In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import openpyxl
import statsmodels



In [2]:
# get the old data sets back that I accidentally deleted

In [3]:
stats_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/ufc_fight_stats.csv'

# data source: https://www.reddit.com/r/MMA/comments/evbnjd/released_offical_ufc_fight_night_weights/
weight_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/UFC Fight Night Weights.xlsx'

results_path = '/Users/caseymoser/Desktop/UFC Analysis/UFC/ufc_fight_results.csv'

stats_df = pd.read_csv(stats_path)

weight_df = pd.read_excel(weight_path)

results_df = pd.read_csv(results_path)


In [4]:
# Step 1: Split fighters into separate columns
results_df[['Fighter_1', 'Fighter_2']] = results_df['BOUT'].str.split(' vs. ', expand=True)

fighter1_df = results_df.copy()
fighter1_df['FIGHTER'] = fighter1_df['Fighter_1']
fighter1_df['RESULT'] = fighter1_df['OUTCOME'].str[0].map({'W': 'Win', 'L': 'Loss'})
fighter1_df = fighter1_df.drop(columns=['Fighter_1','Fighter_2'])


fighter2_df = results_df.copy()
fighter2_df['FIGHTER'] = fighter2_df['Fighter_2']
fighter2_df['RESULT'] = fighter2_df['OUTCOME'].str[2].map({'W': 'Win', 'L': 'Loss'})
fighter2_df = fighter2_df.drop(columns=['Fighter_1','Fighter_2'])


# Note to self, clean this code so that I first drop fighter_1 and fighter_2, then combine the data sets together/

# Combine both into a single dataframe
results_df_clean = pd.concat([fighter1_df, fighter2_df])

results_df_clean['UFC_EVENT'] = results_df_clean['EVENT'].str.extract(r'(UFC \d+)', expand=False)





In [6]:
weight_df['UFC_EVENT'] = weight_df['EVENT'].str.extract(r'(UFC \d+)', expand=False)
weight_df

weight_df['FIGHTER'] = weight_df['FIGHTER'].str.strip().str.lower()
weight_df['UFC_EVENT'] = weight_df['UFC_EVENT'].str.strip().str.upper()

results_df_clean['FIGHTER'] = results_df_clean['FIGHTER'].str.strip().str.lower()
results_df_clean['UFC_EVENT'] = results_df_clean['UFC_EVENT'].str.strip().str.upper()

merged_weight = pd.merge(
    results_df_clean,
    weight_df,
    on=['FIGHTER', 'UFC_EVENT'],
    how='left', 
    suffixes=('_result', '_weight')
)
results_df_clean

# Drop rows with any NaN values
merged_weight_clean = merged_weight.dropna(subset=['WEIGH IN WEIGHT (lbs)'])

merged_weight_clean

,EVENT_result,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,...,UFC_EVENT,EVENT_weight,WEIGHT CLASS,WEIGH IN WEIGHT (lbs),FIGHT NIGHT WEIGHT (lbs),WEIGHT INCREASE (lbs),PERCENTAGE REGAINED,SEX,Unnamed: 8,REPEATS
17,UFC Fight Night: Lewis vs. Teixeira,Nate Landwehr vs. Morgan Charriere,L/W,Featherweight Bout,KO/TKO,3,0:27,3 Rnd (5-5-5),Blake Grice,Punch to Head At Distance,...,NaN,6- UFC on ESPN 41,Featherweight,145.5,161.4,15.9,10.927835,M,NaN,0.0
128,UFC Fight Night: Machado Garry vs. Prates,Da'Mon Blackshear vs. Alatengheili,W/L,Bantamweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Jason Herzog,Chris Lee 27 - 30.Eric Colon 27 - 30.Derek Cle...,...,NaN,6- UFC on ESPN 41,Bantamweight,134.0,152.4,18.4,13.731343,M,NaN,0.0
146,UFC Fight Night: Emmett vs. Murphy,Josh Emmett vs. Lerone Murphy,L/W,Featherweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Jason Herzog,Ben Cartlidge 47 - 48.Sal D'amato 47 - 48.Eric...,...,NaN,3- UFC on ESPN+ 13,Featherweight,146.0,166.8,20.8,14.246575,M,NaN,0.0
156,UFC Fight Night: Emmett vs. Murphy,Martin Buday vs. Uran Satybaldiev,W/L,Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mike Beltran,Eric Colon 28 - 29.David Lethaby 28 - 29.Tony ...,...,NaN,6- UFC on ESPN 41,Heavyweight,266.0,290.4,24.4,9.172932,M,NaN,0.0
183,UFC Fight Night: Vettori vs. Dolidze 2,Marvin Vettori vs. Roman Dolidze,L/W,Middleweight Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Herb Dean,Mike Bell 46 - 49.Derek Cleary 46 - 49.Sal D'a...,...,NaN,3- UFC on ESPN+ 13,Middleweight,185.5,211.0,25.5,13.746631,M,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,UFC Fight Night 56: Shogun vs Saint Preux,Juliana Lima vs. Nina Nunes,W/L,Women's Strawweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,Guilherme Bravo 28 - 29.Derek Cleary 28 - 29.F...,...,NaN,6- UFC on ESPN 41,Women's Flyweight,125.5,130.2,4.7,3.745020,F,NaN,0.0
13957,UFC Fight Night: Fight for the Troops 3,Amanda Nunes vs. Germaine de Randamie,W/L,Women's Bantamweight Bout,KO/TKO,1,3:56,3 Rnd (5-5-5),Herb Dean,Elbows to Head From Mount,...,NaN,3- UFC on ESPN+ 13,Women's Bantamweight,136.0,148.0,12.0,8.823529,F,NaN,0.0
14079,UFC on FOX: Johnson vs Moraga,Julie Kedzie vs. Germaine de Randamie,L/W,Women's Bantamweight Bout,Decision - Split,3,5:00,3 Rnd (5-5-5),Randy Corley,Terry Boehlke 27 - 30.Lester Griffin 29 - 28.L...,...,NaN,3- UFC on ESPN+ 13,Women's Bantamweight,136.0,148.0,12.0,8.823529,F,NaN,0.0
14156,UFC on FOX: Henderson vs Melendez,Chad Mendes vs. Darren Elkins,W/L,Featherweight Bout,KO/TKO,1,1:08,3 Rnd (5-5-5),Herb Dean,Punch to Head At Distance,...,NaN,3- UFC on ESPN+ 13,Featherweight,145.5,167.4,21.9,15.051546,M,NaN,0.0


In [7]:
#df.groupby('FIGHTER')['SIG.STR. %'].mean()

# Log odds of winning based on how much weight you cut
win_weight_df = merged_weight_clean[['FIGHTER','RESULT','WEIGH IN WEIGHT (lbs)', 'FIGHT NIGHT WEIGHT (lbs)', 'WEIGHT INCREASE (lbs)']]

win_weight_df['RESULT'] = win_weight_df['RESULT'].map({'Win': 1, 'Loss': 0})
win_weight_df
#win_weight_df

/var/folders/rc/yw7v7vhj4l3_vjznh0cwm8wr0000gn/T/ipykernel_93515/2478161064.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win_weight_df['RESULT'] = win_weight_df['RESULT'].map({'Win': 1, 'Loss': 0})


,FIGHTER,RESULT,WEIGH IN WEIGHT (lbs),FIGHT NIGHT WEIGHT (lbs),WEIGHT INCREASE (lbs)
17,nate landwehr,0.0,145.5,161.4,15.9
128,da'mon blackshear,1.0,134.0,152.4,18.4
146,josh emmett,0.0,146.0,166.8,20.8
156,martin buday,1.0,266.0,290.4,24.4
183,marvin vettori,0.0,185.5,211.0,25.5
...,...,...,...,...,...
13453,nina nunes,0.0,125.5,130.2,4.7
13957,germaine de randamie,0.0,136.0,148.0,12.0
14079,germaine de randamie,1.0,136.0,148.0,12.0
14156,darren elkins,0.0,145.5,167.4,21.9
